<a href="https://colab.research.google.com/github/haneenalaa465/CodSoft/blob/main/CodSoft_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>